# Entrega 2
## Limpieza y preprocesado de datos
Este notebook contiene una exploración básica, así como operaciones de limpieza y preprocesado de datos del archivo train.csv.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

from google.colab import files
import os

In [ ]:
# Cargar json
files.upload()

In [ ]:
# Cargar competencia de Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

In [ ]:
# descomprimir archivos
!unzip udea*.zip > /dev/null
!wc *.csv

In [ ]:
# Cargar archivo train.csv
file = "train.csv"
df = pd.read_csv(file)
df.head()

## Preprocesado de datos
### 1. Imputación de Datos Faltantes

In [ ]:
# Separar las columnas numéricas y categóricas
columnas_numericas = df.select_dtypes(include=["float64", "int64"]).columns
columnas_categoricas = df.select_dtypes(include=["object"]).columns

# Imputar columnas numéricas usando KNNImputer
imputar_numericas = KNNImputer(n_neighbors=5)
df[columnas_numericas] = imputar_numericas.fit_transform(df[columnas_numericas])

# Imputar columnas categóricas con la moda (más frecuente)
imputar_categoricas = SimpleImputer(strategy="most_frequent")
df[columnas_categoricas] = imputar_categoricas.fit_transform(df[columnas_categoricas])

# Verificar si quedan valores faltantes
print("Valores faltantes después de la imputación:")
print(df.isnull().sum())

### 2. Conversión de Variables Categóricas a One-Hot Encoding

In [ ]:
# Convertir columnas categóricas en variables dummy (One-Hot Encoding)
df_codificado = pd.get_dummies(df, drop_first=True)

# Imprimir df con variables categóricas codificadas
print(df_codificado.head())

### 3. Escalado de Valores Numéricos



In [ ]:
# Escalado con StandardScaler
scaler = StandardScaler()
df_codificado[columnas_numericas] = scaler.fit_transform(
    df_codificado[columnas_numericas]
)

print("Variables numéricas escaladas:")
print(df_codificado[columnas_numericas].head())

### 4. Detectar valores atípicos con Z-score y reemplazarlos por la mediana



In [ ]:
# Definir umbral
umbral = 3

# Aplicar Z-score y reemplazar valores atípicos por la mediana
for col in columnas_numericas:
    if df_codificado[col].std() == 0:
        continue
    z_scores = zscore(df_codificado[col])
    mediana = df_codificado[col].median()
    df_codificado[col] = np.where(
        np.abs(z_scores) > umbral, mediana, df_codificado[col]
    )

print("Valores atípicos suavizados reemplazando por la mediana usando Z-score.")